# Analysis of ALMA NGC 253 Data for Density Tracing Species

## (1) Importing Python packages you will be using

In [40]:
#Use this window to import packages you will use later
#When you first start working or import a new package, this cell needs to be run before doing anything else.


import numpy as np                          # I use this for maths and sometimes arrays.
                                            # "np" is just an abbreviation since we call this package so often!
import pylab                                # I use this for arrays and plotting
import matplotlib                           # Another plotting package
import matplotlib.gridspec as gridspec      # If there is a task you use a lot, importing it like this 
                                            # keeps you from having to constantly type "matplotlib.gridspec"
                                            # every time you call that task!
import scipy

import astropy.io.fits as fits              # I use this to import FITS images. 
                                            # Astropy should be included in your conda installation
from astropy.wcs import WCS                 # This one helps us add coordinates to images
from astropy import units as u              # This one helps with unit conversion
from astropy import constants as const

import regions

import pyspeckit as psk                     # I use this to do spectral line fitting
                                            # You probably don't have it installed; 
                                            # to install, type 'pip install pyspeckit' in a terminal window
        
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes

from reproject import reproject_interp      # Reproject is another useful package you should install
from reproject.mosaicking import find_optimal_celestial_wcs 


                                            # Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

                                            # Super important!! 
                                            # This embeds plots in the Jupyter window 
                                            # (instead of showing them as pop-ups)
%matplotlib inline                             

matplotlib.rc('font', family='sans-serif')  # Set plot fonts
matplotlib.rc('text', usetex=True)      
#matplotlib.rc('text', usetex=False)        # This is a quick fix if you don't have latex on your computer

In [41]:
#path = '/Users/ashleylieber/MIlls_Research/NGC253_AlmaData/HCN4-3.fits'
path = '/Users/ashleylieber/MIlls_Research/NGC253_AlmaData/HCN4-3.fits'
                                # This string is not just the name of a file, but its full address.
                                # If, like this file, it is in the same directory as your notebook,
                                # you can specify just the filename. If it lives elsewhere you need
                                # to give a relative (e.g., data/name.fits or ../directory/name.fits)
                                # or absolute (e.g., EACMills/Project/Data/name.fits) path to the file.


                                # The fits images we will work with consist of two main parts:
                                # (1) An array of 2-4 dimensions that holds the value of each image pixel 
                                # (2) A "header" which contains a summary of information associated with this array                
          

image = fits.getdata(path)      # This command reads in the data array part of the fits file, so we can manipulate it 
                                # like any other python array
    
header = fits.getheader(path)   # This command reads in the header, which consists of a series of variables 
                                # and associated values  

In [42]:
header

SIMPLE  =                    T /Standard FITS                                   
BITPIX  =                  -32 /Floating point (32 bit)                         
NAXIS   =                    3                                                  
NAXIS1  =                 1800                                                  
NAXIS2  =                 1536                                                  
NAXIS3  =                   81                                                  
EXTEND  =                    T                                                  
BSCALE  =   1.000000000000E+00 /PHYSICAL = PIXEL*BSCALE + BZERO                 
BZERO   =   0.000000000000E+00                                                  
BMAJ    =   4.861111111111E-05                                                  
BMIN    =   4.861111111111E-05                                                  
BPA     =   0.000000000000E+00                                                  
BTYPE   = 'Intensity'       

In [5]:
hi_data = fits.open(path)  # Open the FITS file for reading
cube = SpectralCube.read(hi_data)  # Initiate a SpectralCube
hi_data.close() 

In [6]:
print(cube)

SpectralCube with shape=(81, 1536, 1800) and unit=Jy / beam:
 n_x:   1800  type_x: RA---SIN  unit_x: deg    range:    11.878802 deg:   11.897042 deg
 n_y:   1536  type_y: DEC--SIN  unit_y: deg    range:   -25.295373 deg:  -25.281302 deg
 n_s:     81  type_s: FREQ      unit_s: Hz     range: 353981631172.400 Hz:354454632371.123 Hz


## Attempts to fix frequency - velocity issues

In [28]:
# Attempt to adjust Frequency to velcoity issue with equation
restfrq, freqdelta = header['RESTFRQ'], header['CDELT3']
c = const.c.to(u.km/u.s)
adj_velocity = (c * freqdelta) / restfrq
print(adj_velocity)

5.000000000070757 km / s


In [31]:
# Using Spectral cube to obtain the velocity range from the third axis
# Reference webpage: https://spectral-cube.readthedocs.io/en/latest/manipulating.html
rest_frq = cube.header['RESTFRQ']
cube2 = cube.with_spectral_unit(u.km / u.s, velocity_convention='radio',rest_value=rest_frq*u.Hz)
print(cube2)

SpectralCube with shape=(81, 1536, 1800) and unit=Jy / beam:
 n_x:   1800  type_x: RA---SIN  unit_x: deg    range:    11.878802 deg:   11.897042 deg
 n_y:   1536  type_y: DEC--SIN  unit_y: deg    range:   -25.295373 deg:  -25.281302 deg
 n_s:     81  type_s: VRAD      unit_s: km / s  range:       43.000 km / s:     443.000 km / s


In [27]:
cube2.header

SIMPLE  =                    T /Standard FITS                                   
BITPIX  =                  -32 /Floating point (32 bit)                         
NAXIS   =                    3                                                  
NAXIS1  =                 1800                                                  
NAXIS2  =                 1536                                                  
NAXIS3  =                   81                                                  
EXTEND  =                    T                                                  
BSCALE  =   1.000000000000E+00 /PHYSICAL = PIXEL*BSCALE + BZERO                 
BZERO   =   0.000000000000E+00                                                  
BMAJ    =   4.861111111111E-05                                                  
BMIN    =   4.861111111111E-05                                                  
BPA     =   0.000000000000E+00                                                  
BTYPE   = 'Intensity'       

In [18]:
cube.header

SIMPLE  =                    T /Standard FITS                                   
BITPIX  =                  -32 /Floating point (32 bit)                         
NAXIS   =                    3                                                  
NAXIS1  =                 1800                                                  
NAXIS2  =                 1536                                                  
NAXIS3  =                   81                                                  
EXTEND  =                    T                                                  
BSCALE  =   1.000000000000E+00 /PHYSICAL = PIXEL*BSCALE + BZERO                 
BZERO   =   0.000000000000E+00                                                  
BMAJ    =   4.861111111111E-05                                                  
BMIN    =   4.861111111111E-05                                                  
BPA     =   0.000000000000E+00                                                  
BTYPE   = 'Intensity'       

In [39]:
cube2[50,:,:].quicklook() # show an image  


RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 640x480 with 1 Axes>

In [36]:
w1 = WCS(cube2.header)

In [37]:
w1

WCS Keywords

Number of WCS axes: 3
CTYPE : 'RA---SIN'  'DEC--SIN'  'VRAD'  
CRVAL : 11.88791666667  -25.28833333333  443000.00001866  
CRPIX : 901.0  769.0  1.0  
PC1_1 PC1_2 PC1_3  : 1.0  0.0  0.0  
PC2_1 PC2_2 PC2_3  : 0.0  1.0  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : -9.166666666667e-06  9.166666666667e-06  -5000.0000000708  
NAXIS : 1800  1536  81

In [38]:
fig1 = pylab.figure(1,figsize=(10,10))       # We will make this figure #1 and give it a size of 10x10

                                             # We also want to make sure that our image is plotted with sky coordinates
ax1 = pylab.subplot(projection=w1)            # We do this by specifying a projection (the wcs from the previous cell)
RA = ax1.coords[0]
Dec = ax1.coords[1]

im1 = pylab.imshow(image_2D,cmap='Greys')   # We have a lot of control over how we plot the image.
                                            # We can change the color, range of values, even transparency:
#im1 = pylab.imshow(image_2D,cmap='Greys_r')
#im1 = pylab.imshow(image_2D,cmap='bone',vmin=0,vmax=0.002)
#im1 = pylab.imshow(image_2D,cmap='viridis', vmin=0, alpha=0.5)

                                            # Note that each of these options will plot on top of any 
                                            # previously-displayed image. To display an image as a
                                            # new figure, you need to define a new figure command, 
                                            # with a unique figure number, e.g, fig2 = pylab.figure(2)
                                            
                                            # The options below show other ways to customize this figure 
RA.set_ticks(size=-3)                       # Change the length of the tick marks 
Dec.set_ticks(size=-3)                      # (negative values place ticks inside the bounding box)
pylab.xlabel('Right Ascension',fontsize=20,labelpad=1) # Label the x-axis
pylab.ylabel('Declination',fontsize=20,labelpad=0) # Label the y-axis
ax1.tick_params(axis = 'both', which = 'major', labelsize = 15) # Increase the font of the tick labels
pylab.annotate(s='Continuum',fontsize=40,xy=(0.02,0.91),xycoords="axes fraction") # Add a label in the figure
cb=pylab.colorbar(im1,fraction=0.046,pad=0.04) # Define a color bar
cb.set_label(label='Flux Density (Jy / beam)',fontsize=25,rotation=270,labelpad=30) # Add a colorbar label
cb.ax.tick_params(which = 'major', labelsize = 20) # Increase the font size on the colorbar tick labels




ValueError: WCS has more than 2 pixel dimensions, so 'slices' should be set

<Figure size 1000x1000 with 0 Axes>